In [1]:
Pkg.checkout("MyMatching", "master")

INFO: Checking out MyMatching master...
INFO: Pulling MyMatching latest master...
INFO: No packages to install, update or remove


In [2]:
using MyMatching

In [3]:
Pkg.test("MyMatching")

INFO: Testing MyMatching


Test Summary:               | Pass  Total
Testing deferred acceptance |   18     18


INFO: MyMatching tests passed


In [4]:
using MatchingMarkets

In [5]:
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [10]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(7, 4))

(Array{Int64,1}[[4, 3], [2, 3, 4, 1], [3, 1], [1], [1, 3], [4, 3, 1], [1, 2, 3]], Array{Int64,1}[[6, 2, 5, 4, 1], [3, 6, 5, 2, 7, 1, 4], [2, 3], [5, 3]])

In [11]:
p_caps = [1, 1, 2, 1, 1, 2, 1]
r_caps = [3, 3, 1, 2];

In [12]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

([0, 2, 3, 0, 1, 1, 1, 0, 2], [6, 5, 4, 0, 7, 2, 3, 0, 0], [1, 2, 3, 5, 6, 7, 9, 10], [1, 4, 7, 8, 10])

In [3]:
A = zeros(Int, 2, 3)

2×3 Array{Int64,2}:
 0  0  0
 0  0  0

In [22]:
B = fill([0],(2,2))

2×2 Array{Array{Int64,1},2}:
 [0]  [0]
 [0]  [0]

## 例1
```
In [4]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[4]:
(Array{Int64,1}[[2, 1, 3], [3, 1, 2], [1, 3, 2], [1, 3, 2], [1, 2, 3]], Array{Int64,1}[[1, 3, 2, 5, 4], [5, 1, 3, 2, 4], [3, 5, 2, 4, 1]])

In [5]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [2, 2, 3];

In [6]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[6]:
([2, 3, 1, 3, 0, 1, 2], [5, 3, 5, 1, 0, 4, 2], [1, 2, 3, 4, 6, 8], [1, 3, 5, 8])
```
受入側の枠が提案側の枠の分だけあっても選好次第でマッチングが成立しない。

## 例2
```
In [7]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[7]:
(Array{Int64,1}[[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 1, 3], [2, 1, 3]], Array{Int64,1}[[2, 1, 5, 4, 3], [4, 2, 1, 5, 3], [3, 2, 4, 1, 5]])

In [8]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [2, 2, 3];

In [9]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[9]:
([1, 1, 3, 2, 3, 2, 3], [2, 1, 4, 5, 5, 4, 3], [1, 2, 3, 4, 6, 8], [1, 3, 5, 8])
```

## 例3
```
In [10]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(7, 4))

Out[10]:
(Array{Int64,1}[[4, 3], [2, 3, 4, 1], [3, 1], [1], [1, 3], [4, 3, 1], [1, 2, 3]], Array{Int64,1}[[6, 2, 5, 4, 1], [3, 6, 5, 2, 7, 1, 4], [2, 3], [5, 3]])

In [11]:
p_caps = [1, 1, 2, 1, 1, 2, 1]
r_caps = [3, 3, 1, 2];

In [12]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[12]:
([0, 2, 3, 0, 1, 1, 1, 0, 2], [6, 5, 4, 0, 7, 2, 3, 0, 0], [1, 2, 3, 5, 6, 7, 9, 10], [1, 4, 7, 8, 10])
```

In [115]:
function strategic_pref(m_prefs::Vector{Vector{Int}},
                        f_prefs::Vector{Vector{Int}},
                        m_matched::Vector{Int},
                        f_matched::Vector{Int},
                        p_count::Vector{Int},
                        false_prefs::Vector{Vector{Int}})
    m = length(m_prefs)
    n = length(f_prefs)
    results = fill([0], (n, factorial(m)))
    f_require = zeros(Int64, n)
    for i in 1:n
        if f_matched[i] == f_prefs[i][1]
            f_require[i] = 1
        end
        if p_count[i] <= 1
            f_require[i] = 1
        end
    end

    for i in 1:n
        if f_require[i] == 0
            f_new_prefs = copy(f_prefs)
            for j in 1:factorial(m)
                f_new_prefs[i] = false_prefs[j]
                m_new_matched, f_new_matched = my_deferred_acceptance(m_prefs, f_new_prefs)
                if findfirst(f_prefs, f_new_matched[i]) < findfirst(f_prefs, f_matched[i])
                    results[i, j] = false_prefs[j]
                end
            end
        end
    end
    return results
end

strategic_pref (generic function with 1 method)

In [152]:
m_prefs, f_prefs = mat2vecs.(random_prefs(6, 6, allow_unmatched = false))

(Array{Int64,1}[[6, 4, 1, 2, 5, 3], [4, 1, 5, 3, 6, 2], [4, 5, 6, 2, 1, 3], [1, 5, 3, 6, 2, 4], [1, 2, 5, 4, 6, 3], [5, 2, 4, 6, 3, 1]], Array{Int64,1}[[4, 2, 3, 6, 5, 1], [2, 1, 5, 6, 4, 3], [6, 5, 3, 4, 1, 2], [5, 6, 3, 2, 1, 4], [4, 2, 1, 5, 3, 6], [5, 3, 4, 2, 1, 6]])

In [153]:
m_matched, f_matched, p_count = my_deferred_acceptance(m_prefs, f_prefs)

([0, 5, 6, 1, 2, 4], [4, 5, 0, 6, 2, 3], [3, 2, 0, 3, 3, 2])

In [148]:
using Combinatorics

In [149]:
false_prefs = collect(permutations([1,2,3,4,5,6]))

720-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6]
 [1, 2, 3, 4, 6, 5]
 [1, 2, 3, 5, 4, 6]
 [1, 2, 3, 5, 6, 4]
 [1, 2, 3, 6, 4, 5]
 [1, 2, 3, 6, 5, 4]
 [1, 2, 4, 3, 5, 6]
 [1, 2, 4, 3, 6, 5]
 [1, 2, 4, 5, 3, 6]
 [1, 2, 4, 5, 6, 3]
 [1, 2, 4, 6, 3, 5]
 [1, 2, 4, 6, 5, 3]
 [1, 2, 5, 3, 4, 6]
 ⋮                 
 [6, 5, 3, 1, 2, 4]
 [6, 5, 3, 1, 4, 2]
 [6, 5, 3, 2, 1, 4]
 [6, 5, 3, 2, 4, 1]
 [6, 5, 3, 4, 1, 2]
 [6, 5, 3, 4, 2, 1]
 [6, 5, 4, 1, 2, 3]
 [6, 5, 4, 1, 3, 2]
 [6, 5, 4, 2, 1, 3]
 [6, 5, 4, 2, 3, 1]
 [6, 5, 4, 3, 1, 2]
 [6, 5, 4, 3, 2, 1]

In [150]:
results = strategic_pref(m_prefs, f_prefs, m_matched, f_matched, p_count, false_prefs)

6×720 Array{Array{Int64,1},2}:
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]  …  [0]  [0]  [0]  [0]  [0]  [0]  [0]
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]     [0]  [0]  [0]  [0]  [0]  [0]  [0]
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]     [0]  [0]  [0]  [0]  [0]  [0]  [0]
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]     [0]  [0]  [0]  [0]  [0]  [0]  [0]
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]     [0]  [0]  [0]  [0]  [0]  [0]  [0]
 [0]  [0]  [0]  [0]  [0]  [0]  [0]  [0]  …  [0]  [0]  [0]  [0]  [0]  [0]  [0]

In [151]:
print(results)

Array{Int64,1}[[0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [0] [